In [1]:
import requests
from urllib import request
from bs4 import BeautifulSoup
import time
import pandas as pd
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import time
import sqlite3
from requests import Session
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import datetime
from tqdm import tqdm


In [22]:
ACODE = {"호텔신라":"008770","두산퓨얼셀":"336260","한화솔루션":"009830",
            "에코프로":"086520","신성이엔지":"011930","유니슨":"018000",
            "카카오":"035720","네이버":"035420","엔씨소프트":"036570",
            "sds":"018260","삼성전자":"005930","SK하이닉스":"000660",
            "skc":"011790","한솔케미칼":"014680","DB하이텍":"000990",
            "롯데쇼핑":"023530","신세계":"004170","삼성물산":"028260",
            "동서":"026960","kb금융":"105560","한국금융지주":"071050","신한지주":"055550",
            "우리금융지주":"316140","하나금융지주":"086790",
         "넥슨지티":"041140","한빛소프트":"047080","코세스":"089890",
      "드래곤플라이":"030350","에이트원":"230980","엔텔스":"069410"}

In [23]:
DATE_STOP = '2021-12-23'

In [24]:
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params=params if method.upper() == 'GET' else '',
                               data=params if method.upper() == 'POST' else '',
                               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60)  # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit - 1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)

    return resp

In [33]:
def naver_crawl(ACODE):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
    pages = [1]
    title_result=[]
   
    global DATE_STOP
    #seen_date = db.naver_title_crawl_1114.find({"code":acode}).sort('date', -1).limit(1)
    
    #sort 전 어떤 변수에 저장해서, 
    #코드 기준으로 가져와서 데이트 정렬
    #seen_date = list(seen_date)
    #if not seen_date:
        
    seen_date = "2021.12.23 00:00"
    #TF = False

    for page in pages:
            #title_result=[]
        pages.append(page + 1)
        url = 'https://finance.naver.com/item/board.nhn?code={}&page={}'.format(ACODE,page)
        resp = download(url, headers = headers)
        dom = BeautifulSoup(resp.content, 'lxml')
        table = dom.find('table', {'class' : 'type2'})
        tt = table.select('tbody > tr')
        for k in tt[2:]:
            if len(k.select('td > span')) > 0:
                date = k.select('td > span')[0].text
                if str(date)<=str(seen_date) or date[:10]==DATE_STOP:
                    TF = True
                    break

                title = k.select('td.title > a')[0]['title']
                views = k.select('td > span')[1].text
                pos = k.select('td > strong')[0].text
                neg = k.select('td > strong')[1].text
                # print(title)

                put_data = {
                                     'code':ACODE,
                                     'date':date,
                                      'title':title,
                                      'views':views,
                                      'pos':pos,
                                      'neg':neg
                            }
                # print(put_data)
                title_result.append(put_data)
                
        #if TF:
            #break
        return title_result
        #else:
        # print(title_result)

In [35]:
import time
import random
import datetime
from kafka import KafkaProducer

bootstrap_servers = ['localhost:9093']  # kafka broker ip
topicName = 'NAVER'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)


for key,value in ACODE.items():
    result = naver_crawl(value)
    for i in result:
        print(i)
        producer.send(topicName, str(i).encode())

        tim = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        producer.send(topicName, tim.encode())

{'code': '008770', 'date': '2021.12.27 21:32', 'title': '호주는 앞서 간다', 'views': '182', 'pos': '0', 'neg': '1'}
{'code': '008770', 'date': '2021.12.27 19:17', 'title': '더 떨어지면 좋겠다', 'views': '158', 'pos': '0', 'neg': '2'}
{'code': '008770', 'date': '2021.12.27 17:59', 'title': '공매.  강아지들 안티아닌척 글쓰네', 'views': '110', 'pos': '0', 'neg': '0'}
{'code': '008770', 'date': '2021.12.27 16:55', 'title': '더 강려크한 코로나가 올지', 'views': '153', 'pos': '0', 'neg': '1'}
{'code': '008770', 'date': '2021.12.27 16:28', 'title': '공매도 치는사람들', 'views': '161', 'pos': '2', 'neg': '0'}
{'code': '008770', 'date': '2021.12.27 15:12', 'title': '하루종일 뭐하는지 ^^', 'views': '163', 'pos': '0', 'neg': '0'}
{'code': '008770', 'date': '2021.12.27 13:40', 'title': '코로나 끝나야 게임 오버되죠..', 'views': '253', 'pos': '7', 'neg': '0'}
{'code': '008770', 'date': '2021.12.27 12:53', 'title': '일봉 20일선 상향으로 방향전환', 'views': '257', 'pos': '4', 'neg': '0'}
{'code': '008770', 'date': '2021.12.27 12:25', 'title': '선생님들께서 뭔가 오해를 하고 계신듯 한데요', 'views': 

{'code': '011930', 'date': '2021.12.28 03:15', 'title': '또 bw 발행하면 주총때 용감하게 맞서자', 'views': '67', 'pos': '0', 'neg': '0'}
{'code': '011930', 'date': '2021.12.28 02:07', 'title': '주린이들을위해~  저도주린이^^!', 'views': '153', 'pos': '2', 'neg': '1'}
{'code': '011930', 'date': '2021.12.27 22:16', 'title': '배터리 소재 재활용 2050 탄소중립', 'views': '439', 'pos': '6', 'neg': '7'}
{'code': '011930', 'date': '2021.12.27 21:59', 'title': '경영진 전원 사퇴해라', 'views': '98', 'pos': '15', 'neg': '0'}
{'code': '011930', 'date': '2021.12.27 20:44', 'title': '1900', 'views': '102', 'pos': '4', 'neg': '3'}
{'code': '011930', 'date': '2021.12.27 20:06', 'title': '18  또BW야?', 'views': '174', 'pos': '6', 'neg': '2'}
{'code': '011930', 'date': '2021.12.27 19:01', 'title': '야 잡주야', 'views': '115', 'pos': '4', 'neg': '1'}
{'code': '011930', 'date': '2021.12.27 17:17', 'title': '왜?', 'views': '251', 'pos': '4', 'neg': '12'}
{'code': '011930', 'date': '2021.12.27 16:52', 'title': '500원', 'views': '239', 'pos': '6', 'neg': '5'}
{'cod

{'code': '036570', 'date': '2021.12.28 07:56', 'title': '도박장 끝이보인다 ㅋㅋ오늘도 빅새일이냐 ㅋㅋ', 'views': '12', 'pos': '0', 'neg': '0'}
{'code': '036570', 'date': '2021.12.28 07:39', 'title': '내일은 공매 타임이다', 'views': '27', 'pos': '0', 'neg': '0'}
{'code': '036570', 'date': '2021.12.28 07:37', 'title': '앞으로 최소한달은 이가격', 'views': '25', 'pos': '0', 'neg': '0'}
{'code': '036570', 'date': '2021.12.28 07:04', 'title': '리니지W, 더욱 와이드해진다', 'views': '84', 'pos': '2', 'neg': '0'}
{'code': '036570', 'date': '2021.12.28 06:20', 'title': '오늘자 뉴스입니다. 참조하실분 참조하세요', 'views': '105', 'pos': '0', 'neg': '1'}
{'code': '036570', 'date': '2021.12.28 04:42', 'title': '엔씨가 어떤회산대', 'views': '109', 'pos': '0', 'neg': '0'}
{'code': '036570', 'date': '2021.12.28 04:13', 'title': '이정도면 사기게임아니냐? 주작 오지네', 'views': '116', 'pos': '0', 'neg': '2'}
{'code': '036570', 'date': '2021.12.28 04:03', 'title': '미증 상승하네', 'views': '111', 'pos': '1', 'neg': '0'}
{'code': '036570', 'date': '2021.12.28 03:23', 'title': '리니지2w 나온다.', 'views': '180

{'code': '011790', 'date': '2021.12.28 07:56', 'title': 'SKC', 'views': '11', 'pos': '1', 'neg': '0'}
{'code': '011790', 'date': '2021.12.28 05:17', 'title': 'ETF지분 확대준비', 'views': '167', 'pos': '1', 'neg': '2'}
{'code': '011790', 'date': '2021.12.28 05:08', 'title': '2차전지', 'views': '184', 'pos': '2', 'neg': '2'}
{'code': '011790', 'date': '2021.12.28 04:23', 'title': '챠트가 영 엉망이네', 'views': '119', 'pos': '3', 'neg': '1'}
{'code': '011790', 'date': '2021.12.28 00:44', 'title': '테슬라 리비안', 'views': '189', 'pos': '2', 'neg': '2'}
{'code': '011790', 'date': '2021.12.27 23:15', 'title': '오늘밤에', 'views': '325', 'pos': '3', 'neg': '3'}
{'code': '011790', 'date': '2021.12.27 21:28', 'title': '평단 177,272 ㅜㅜ', 'views': '327', 'pos': '1', 'neg': '2'}
{'code': '011790', 'date': '2021.12.27 21:23', 'title': '이제', 'views': '361', 'pos': '7', 'neg': '1'}
{'code': '011790', 'date': '2021.12.27 20:00', 'title': '징하다', 'views': '223', 'pos': '5', 'neg': '1'}
{'code': '011790', 'date': '2021.12.27 19:54'

{'code': '004170', 'date': '2021.12.27 17:44', 'title': '좃세계 버리고 곱버스몰빵한결과', 'views': '281', 'pos': '4', 'neg': '4'}
{'code': '004170', 'date': '2021.12.27 17:03', 'title': '신세계백화점 관련 기사 올려드려요', 'views': '137', 'pos': '0', 'neg': '0'}
{'code': '004170', 'date': '2021.12.27 15:45', 'title': '고수님들', 'views': '151', 'pos': '0', 'neg': '0'}
{'code': '004170', 'date': '2021.12.27 13:21', 'title': '거래량이 너무 적다', 'views': '123', 'pos': '2', 'neg': '3'}
{'code': '004170', 'date': '2021.12.27 11:23', 'title': '홀짝홀짝 챠트네', 'views': '164', 'pos': '2', 'neg': '0'}
{'code': '004170', 'date': '2021.12.27 09:33', 'title': '고용노동부가 감사에 착수했습니다 올 세무사 시험의 불공정성을', 'views': '339', 'pos': '2', 'neg': '7'}
{'code': '004170', 'date': '2021.12.27 07:46', 'title': '요즘 인싸들', 'views': '421', 'pos': '4', 'neg': '2'}
{'code': '004170', 'date': '2021.12.26 23:50', 'title': '주주로서 명동본점 자랑스럽다', 'views': '224', 'pos': '0', 'neg': '1'}
{'code': '004170', 'date': '2021.12.26 23:03', 'title': '평단 29 -15% 손절침', 'views': '562', 

{'code': '071050', 'date': '2021.12.28 07:47', 'title': '■ 올해 경영진들이 보여준 주주 권익 수준?', 'views': '13', 'pos': '0', 'neg': '0'}
{'code': '071050', 'date': '2021.12.28 07:05', 'title': '배당성향 30% 위로 높이고 자사주 매입 및 소각 하라', 'views': '30', 'pos': '2', 'neg': '0'}
{'code': '071050', 'date': '2021.12.28 01:00', 'title': '내년3월까지들고갈 형들', 'views': '124', 'pos': '9', 'neg': '0'}
{'code': '071050', 'date': '2021.12.28 00:15', 'title': '신영증권도 배당금 4000', 'views': '89', 'pos': '1', 'neg': '0'}
{'code': '071050', 'date': '2021.12.27 22:37', 'title': '20만 고지 출정가.', 'views': '122', 'pos': '2', 'neg': '1'}
{'code': '071050', 'date': '2021.12.27 21:59', 'title': '2021년 12월 27일 월요일 시황분석', 'views': '100', 'pos': '2', 'neg': '1'}
{'code': '071050', 'date': '2021.12.27 18:53', 'title': '한금지. 기로의 순간이다', 'views': '150', 'pos': '1', 'neg': '2'}
{'code': '071050', 'date': '2021.12.27 17:20', 'title': '남구가 esg경영 잘하구먼', 'views': '152', 'pos': '8', 'neg': '0'}
{'code': '071050', 'date': '2021.12.27 16:51', 'title': '증권사별 배

{'code': '041140', 'date': '2021.12.28 07:15', 'title': '병합은 뭐다? 병합 같일하는 재료 가격이다', 'views': '22', 'pos': '0', 'neg': '0'}
{'code': '041140', 'date': '2021.12.28 06:04', 'title': '현시간 중공군이 삼팔서늘넘어 처드러오고?는대', 'views': '37', 'pos': '0', 'neg': '1'}
{'code': '041140', 'date': '2021.12.28 05:07', 'title': '장전부터 미친드시 하락하고?는대 이유가 머나', 'views': '56', 'pos': '0', 'neg': '0'}
{'code': '041140', 'date': '2021.12.28 04:13', 'title': '미국 3대지수 폭등중', 'views': '120', 'pos': '2', 'neg': '0'}
{'code': '041140', 'date': '2021.12.28 06:21', 'title': '야이 ?거?너마 미구기오른다고 올라', 'views': '21', 'pos': '0', 'neg': '1'}
{'code': '041140', 'date': '2021.12.28 02:20', 'title': '탈 난다', 'views': '65', 'pos': '0', 'neg': '1'}
{'code': '041140', 'date': '2021.12.28 00:11', 'title': '이걸사서 넷게임즈로 바꾸면 돈 되네', 'views': '163', 'pos': '2', 'neg': '0'}
{'code': '041140', 'date': '2021.12.27 22:16', 'title': '진짜 개미들 겁나 털렸네요', 'views': '250', 'pos': '0', 'neg': '0'}
{'code': '041140', 'date': '2021.12.27 21:59', 'title': '●●●●●', 'v

{'code': '230980', 'date': '2021.12.24 21:21', 'title': '에이트원 적정가', 'views': '935', 'pos': '4', 'neg': '3'}
{'code': '230980', 'date': '2021.12.24 15:01', 'title': '♥뉴욕경험및 영어메타버스', 'views': '346', 'pos': '2', 'neg': '2'}
{'code': '230980', 'date': '2021.12.23 22:15', 'title': '해석요망', 'views': '1243', 'pos': '5', 'neg': '2'}
{'code': '230980', 'date': '2021.12.24 15:43', 'title': '해석요망', 'views': '122', 'pos': '0', 'neg': '1'}
{'code': '230980', 'date': '2021.12.23 17:53', 'title': '내가 왜 이글 쓰냐며는....', 'views': '855', 'pos': '8', 'neg': '4'}
{'code': '230980', 'date': '2021.12.23 17:18', 'title': '이주시기는 내년까지도 만사가 꼬일주시기여.', 'views': '556', 'pos': '4', 'neg': '2'}
{'code': '230980', 'date': '2021.12.23 15:19', 'title': '흠', 'views': '482', 'pos': '1', 'neg': '0'}
{'code': '230980', 'date': '2021.12.23 11:11', 'title': '진짜 그지같네 ', 'views': '420', 'pos': '7', 'neg': '1'}
{'code': '069410', 'date': '2021.12.27 16:16', 'title': '이 썩을회사', 'views': '68', 'pos': '1', 'neg': '0'}
{'code': '069410'